In [1]:
%matplotlib inline
from __future__ import division
import numpy as np
import os
import sys
import datetime
from subprocess import call
import subprocess
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import djPyi2 as DJ
from djPyi2 import Common as CM
from djPyi2 import mpltools

import pandas as pd
pd.options.mode.chained_assignment = None
import csv
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy 
import pybedtools as pbt
import ciepy
import cardipspy as cpy
import itertools
import tempfile
import six
import networkx as nx
import scipy.stats as stats
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
from mpl_toolkits.axes_grid1 import  make_axes_locatable
import datetime

from scipy.stats import mode
dy_name = 'categorize_sv_wrt_genes'

private_out = os.path.join(DJ.root, 'private_output', dy_name)
if not os.path.exists(private_out):
    cpy.makedir(private_out)

import gc

In [2]:
def categorize_intersect(intersect_types):
    """ categorize the variant into one genic type, based on types present for a variant"""
    
    if len(intersect_types) == 1:
        out = intersect_types.pop()
        if out == 'intersects_gene':
            return 'intersects_intron'
        else:
            return out
    hierarchy = ['contains_gene', 'contains_exon', 'intersects_exon', 'intersects_promoter']    
    for i in hierarchy:
        if i in intersect_types:
            return i
    else:
        return 'missed case'
def intersect_and_categorize_loc(bt1, bt2, tx_gene_dict):
    """ process the intersect with genes/exons/promoters from gencode
    output mutually exclusive categories: contains gene > contains exon > intersects exon > 
    intersects promoter > intergenic  
    
    count these for all categories specified in fifth column of bedfile """
    
    intersect = bt1.intersect(bt2, wo=True)
    # read the intersect line by line and count up the intersect_scenarios
    out_intersect_data = []
    categorized_all = []
    with open(intersect.fn, 'r') as F:
        count = 0
        var_count = 1
        ID_current = False
        intersect_types = []
        for line in F:
            contains = False
            line = line.rstrip() 
            lin_spl = line.split()
            
            # start end of A
        
            ID_A = lin_spl[3]
            ID_B = lin_spl[8]
      
            if count == 0:
                ID_current = copy.deepcopy(ID_A)
                intersect_types = []
                
            if ID_A != ID_current:
                if len(intersect_types) > 0:
                    var_count +=1
                    category = categorize_intersect(set(intersect_types))
                    out_intersect_data.append([ID_A, category, category_A])
    
                else:
                    print intersect_types, ID_A, category_A
                    break 
                ID_current = copy.deepcopy(ID_A)
                intersect_types = []
            
            category_A = lin_spl[4]    
            start, end = int(lin_spl[1]), int(lin_spl[2])
            length_A = end - start
            
            start, end = int(lin_spl[6]), int(lin_spl[7])
            length_B = end - start
            
            
            
            category_B = lin_spl[11]
            overlap = int(lin_spl[-1])
            
            if overlap >= length_B:
                out_type = "{}_{}".format('contains', category_B)
                intersect_types.append(out_type)
            else:
                out_type = "{}_{}".format('intersects', category_B)
                intersect_types.append(out_type)  
                
            gene_mod = ID_B.split('_')[0]
            categorized_all.append([count, ID_A, ID_B, gene_mod, out_type, overlap])
            count +=1
            
    df_all = pd.DataFrame(categorized_all, columns = ['index', 'ID', 'feature', 
                                                      'feature_mod', 'genic_category_indiv', 'overlap'])
    
    
    df_all['gene_id'] = df_all.feature_mod.apply(lambda x: tx_gene_dict.get(x, x))
    return intersect, df_all

In [3]:
def prep_tx_info(df):
    df = df.copy()
    df['gene_id_mod'] = df.gene_id.apply(lambda x: x.split('.')[0])
    df = df.set_index('tx_id', drop = False)
    return df

def post_process_intersect(info_all_feat, info_all_vars):
    info_all_feat['gene_id_mod'] = info_all_feat.gene_id.apply(lambda x: x.split('.')[0])
    info_cat_per_gene = info_all_feat.groupby(['ID', 'gene_id_mod']).genic_category_indiv.apply(lambda x: categorize_intersect(list(set(x))))
    info_cat_per_gene = info_cat_per_gene.reset_index()
    
    missing = set(info_all_vars.ID.tolist()).difference(info_all_feat.ID.tolist())
    df = pd.DataFrame(index = missing, columns=['genic_category_indiv'])
    df.loc[missing] = 'intergenic'
    df['ID'] = df.index
    df = df.reset_index(drop=True)
    info_cat_per_gene_all = pd.concat([info_cat_per_gene, df])
    info_cat_per_gene_all['genic_category_variant']= info_cat_per_gene_all['genic_category_indiv']
    info_cat_per_gene_all['unique_id'] = info_cat_per_gene_all.gene_id_mod + '_' + info_cat_per_gene_all.ID
    return info_cat_per_gene_all

def genes_impacted_at_top_level(info_cat_per_gene_all, out_strings = False):
    intersect_types = ['intersects_intron', 'intersects_promoter','contains_gene',
                       'intersects_exon', 'contains_exon']
            
    data = []
    for i, df in info_cat_per_gene_all.groupby('ID'):
        genic_categories = df.genic_category_variant.tolist()
        super_category = categorize_intersect(list(set(genic_categories)))
        if super_category == 'missed case':
            print df, genic_categories
            break
        
        if super_category == 'intergenic':
            g = False
        else:
            g = df[df.genic_category_variant == super_category].gene_id_mod.tolist()
            if out_strings:
                g = ",".join(sorted(g))
        
        
        out_data = [i, super_category, g]
        d = {}
        for ind, x in df.iterrows():
            cat = x['genic_category_variant']
            gene = x['gene_id_mod']
            d[cat] = d.get(cat, []) + [gene]
        
        genes_affected = set()
        for t in intersect_types:
            a = d.get(t, False)
            if a:
                genes_affected.update(set(a))
                if out_strings:
                    a = ",".join(sorted(a))
                    out_data.append(a)
                else:
                    out_data.append(set(a))
            else:
                out_data.append(False)
                
        num_affected = len(genes_affected)
        
        if out_strings:
            genes_affected = ",".join(sorted(list(genes_affected)))
            
        out_data += [genes_affected, num_affected]
        data.append(out_data)
    
    cols = (['ID', 'super_category', 'genes_supercategory'] + intersect_types + 
            ['genes_affected', 'num_genes_affected'])
    df = pd.DataFrame(data, columns= cols )
    return df

In [4]:
def prep_info_all(df): 
    df = df.copy()
    df['chrom'] = df.CHROM.apply(lambda x: "chr{}".format(str(x)))
    return df

In [11]:
dy_name = 'qtl_results_01_17'
outdir = os.path.join(private_out, dy_name)
if not os.path.exists(outdir):
    DJ.makedir(outdir)

In [10]:
dy_bed_files = private_out + '/bed_files'
if not os.path.exists(dy_bed_files):
    DJ.makedir(dy_bed_files)

In [20]:
info_all = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/combined_full_vcf_sv_strs/info_all_pass_nref1_annot_final_414.pkl').pipe(prep_info_all)

In [22]:
fn_vars_bt = dy_bed_files + '/all_vars_nnre1.bed'

cols = ['chrom', 'POS', 'END', 'ID', 'SVTYPE']
vars_bt = pbt.BedTool.from_dataframe(info_all[cols])
vars_bt = vars_bt.sort()
vars_bt = vars_bt.moveto(fn_vars_bt)

In [23]:
fn_vars_bt = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/eqtl_processing/qtl_results_12_5/bed_files/all_vars_nnre1.bed'

In [25]:
tx_info =  pd.read_table('/publicdata/gencode_v19_20151104/transcript_to_gene.tsv', names=['tx_id', 'gene_id']).pipe(prep_tx_info)
tx_gene_dict = tx_info.gene_id_mod.to_dict()

fn = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/enrichment_shuffle_svs/bed_files_features/genic_features_all.bed'
features_bt = pbt.BedTool(fn)

In [26]:
intersect_feat, info_all_feat = intersect_and_categorize_loc(vars_bt, features_bt, tx_gene_dict)
info_cat_per_gene_all =  post_process_intersect(info_all_feat, info_all)

In [31]:
cat_sv_str_overall_sets =  genes_impacted_at_top_level(info_cat_per_gene_all)
cat_sv_str_overall_strings = genes_impacted_at_top_level(info_cat_per_gene_all, out_strings=True)

In [36]:
CM.save_dataframe('info_var_gene_intersects_i2QTL_411_nnref1', info_cat_per_gene_all, outdir)

info_var_gene_intersects_i2QTL_411_nnref1 = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_intersects_i2QTL_411_nnref1.pkl')
info_var_gene_intersects_i2QTL_411_nnref1 = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_intersects_i2QTL_411_nnref1.tsv', sep='\t')
# all vars recorded: /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/load_saved_nb_variables.py
# pickled vars recorded:/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/load_pickled_nb_variables.py


In [34]:
CM.save_dataframe('info_var_gene_wide_string_i2QTL_411_nnref1', cat_sv_str_overall_strings, outdir)

info_var_gene_wide_string_i2QTL_411_nnref1 = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_string_i2QTL_411_nnref1.pkl')
info_var_gene_wide_string_i2QTL_411_nnref1 = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_string_i2QTL_411_nnref1.tsv', sep='\t')
# all vars recorded: /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/load_saved_nb_variables.py
# pickled vars recorded:/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/load_pickled_nb_variables.py


In [35]:
CM.save_dataframe('info_var_gene_wide_sets_i2QTL_411_nnref1', cat_sv_str_overall_sets, outdir)

info_var_gene_wide_sets_i2QTL_411_nnref1 = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_sets_i2QTL_411_nnref1.pkl')
info_var_gene_wide_sets_i2QTL_411_nnref1 = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_sets_i2QTL_411_nnref1.tsv', sep='\t')
# all vars recorded: /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/load_saved_nb_variables.py
# pickled vars recorded:/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/load_pickled_nb_variables.py


# Full annotations for 472 Consented 

In [32]:
dy_name = 'consented_472'
outdir = os.path.join(private_out, dy_name)
if not os.path.exists(outdir):
    DJ.makedir(outdir)

In [3]:
dy_bed_files = private_out + '/bed_files'
if not os.path.exists(dy_bed_files):
    DJ.makedir(dy_bed_files)

In [34]:
info_pass_consented_472 = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/dosage_vcf_and_variant_info/info_pass_consented_472.pkl').pipe(prep_info_all)

In [35]:
fn_vars_bt = dy_bed_files + '/all_vars_consented_nnref1.bed'
cols = ['chrom', 'POS', 'END', 'ID', 'SVTYPE_all']
vars_bt = pbt.BedTool.from_dataframe(info_pass_consented_472[cols])
vars_bt = vars_bt.sort()
vars_bt = vars_bt.moveto(fn_vars_bt)

In [36]:
tx_info =  pd.read_table('/publicdata/gencode_v19_20151104/transcript_to_gene.tsv', names=['tx_id', 'gene_id']).pipe(prep_tx_info)
tx_gene_dict = tx_info.gene_id_mod.to_dict()

fn = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/enrichment_shuffle_svs/bed_files_features/genic_features_all.bed'
features_bt = pbt.BedTool(fn)

In [37]:
intersect_feat, info_all_feat = intersect_and_categorize_loc(vars_bt, features_bt, tx_gene_dict)
info_cat_per_gene_all =  post_process_intersect(info_all_feat, info_pass_consented_472)

In [38]:
cat_sv_str_overall_sets =  genes_impacted_at_top_level(info_cat_per_gene_all)
cat_sv_str_overall_strings = genes_impacted_at_top_level(info_cat_per_gene_all, out_strings=True)

In [39]:
CM.save_dataframe('info_var_gene_intersects_i2QTL_472_nnref1', info_cat_per_gene_all, outdir)

info_var_gene_intersects_i2QTL_472_nnref1 = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/info_var_gene_intersects_i2QTL_472_nnref1.pkl')
info_var_gene_intersects_i2QTL_472_nnref1 = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/info_var_gene_intersects_i2QTL_472_nnref1.tsv', sep='\t')
# all vars recorded: /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/load_saved_nb_variables.py
# pickled vars recorded:/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/load_pickled_nb_variables.py


In [40]:
CM.save_dataframe('info_var_gene_wide_string_i2QTL_472_nnref1', cat_sv_str_overall_strings, outdir)

info_var_gene_wide_string_i2QTL_472_nnref1 = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/info_var_gene_wide_string_i2QTL_472_nnref1.pkl')
info_var_gene_wide_string_i2QTL_472_nnref1 = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/info_var_gene_wide_string_i2QTL_472_nnref1.tsv', sep='\t')
# all vars recorded: /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/load_saved_nb_variables.py
# pickled vars recorded:/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/load_pickled_nb_variables.py


In [41]:
CM.save_dataframe('info_var_gene_wide_sets_i2QTL_472_nnref1', cat_sv_str_overall_sets, outdir)

info_var_gene_wide_sets_i2QTL_472_nnref1 = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/info_var_gene_wide_sets_i2QTL_472_nnref1.pkl')
info_var_gene_wide_sets_i2QTL_472_nnref1 = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/info_var_gene_wide_sets_i2QTL_472_nnref1.tsv', sep='\t')
# all vars recorded: /frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/load_saved_nb_variables.py
# pickled vars recorded:/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/load_pickled_nb_variables.py


# Annotate the SNV/Indel Top hits

In [52]:
def classify_snv_indel(x):
    spl = x.split('_')
    ref = spl[-2]
    alt = spl[-1]
    
    len_ref = len(ref)
    len_alt = len(alt)
    
    if alt == "*":
        return 'INDEL'
    
    elif len_ref == len_alt:
        return 'SNV'
    
    else:
        return 'INDEL'

def classify_indel_change(x):
    spl = x.split('_')
    ref = spl[-2]
    alt = spl[-1]
    
    len_ref = len(ref)
    len_alt = len(alt)
    
    change = (len_alt - len_ref)
    
    if alt == "*":
        return -1 * len_ref
    
    else:
        return change

def classify_snv_indel_subcat(x):
    if x == 0:
        return 'SNV'
    if x > 0:
        return 'INS'
    if x < 0:
        return 'DEL'
    
    return 'missed_case'

In [9]:
dy_name = 'qtl_results_01_17'
outdir = os.path.join(private_out, dy_name)
if not os.path.exists(outdir):
    DJ.makedir(outdir)

In [10]:
top_hits_sv_snp_maf5 = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/eqtl_processing/qtl_results_01_17_v2/top_hits_sv_snp_maf5_story.pkl')

In [103]:
new =  pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/eqtl_processing/qtl_results_01_17_v2/top_hits_sv_snp_maf5_story_v2.pkl')

In [108]:
fn_top_combined = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/eqtl_processing/qtl_data_sv_snp_1_14_19/top_qtl_results_sv_snp_combined.txt'
test = pd.read_table(fn_top_combined)

In [20]:
fn_top_combined = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/eqtl_processing/qtl_data_sv_snp_1_14_19/top_qtl_results_sv_snp_combined.txt'

snv_indel_top_all = pd.read_table(fn_top_combined)

In [12]:
info_all_rna = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/dosage_vcf_and_variant_info/info_unfilt_pass_rna_398.pkl')[['ID', 'NR_ID', 'SVTYPE_NR', 'CALLER', 'SVTYPE']]

In [21]:
snv_indel_top_all = snv_indel_top_all.merge(info_all_rna, left_on='snp_id', right_on='ID', how = 'left')

In [69]:
snv_indel = snv_indel_top_all[snv_indel_top_all.CALLER.isnull() == True]

snv_indel['CHANGE'] = snv_indel.snp_id.apply(lambda x: classify_indel_change(x))
snv_indel['VARIANT_SUBTYPE'] = snv_indel.CHANGE.apply(lambda x: classify_snv_indel_subcat(x))
snv_indel['CALLER'] = 'GATK'

snv_indel['start_variant'] =  snv_indel.snp_id.apply(lambda x: x.split('_')[1]).astype(int)
snv_indel['end_variant'] =  snv_indel.start_variant
snv_indel['VARIANT_TYPE'] = snv_indel.snp_id.apply(lambda x: classify_snv_indel(x))
snv_indel['adjust'] = 0
inds = snv_indel[snv_indel.VARIANT_SUBTYPE == 'DEL'].index.tolist()
snv_indel.loc[inds, 'adjust'] = snv_indel.loc[inds, 'CHANGE'].abs()
snv_indel['end_variant'] = snv_indel.end_variant + snv_indel.adjust

snv_indel['ref'] = snv_indel.snp_id.apply(lambda x: x.split('_')[-2])
snv_indel['alt'] = snv_indel.snp_id.apply(lambda x: x.split('_')[-1])

snv_indel['chrom'] = 'chr' + snv_indel.feature_chromosome.astype(str)

In [77]:
snv_indels_lead  = snv_indel.drop_duplicates(['snp_id'])

In [78]:
fn_vars_bt = dy_bed_files + '/sig_snv_indel_leads_all.bed'
cols = ['chrom', 'start_variant', 'end_variant', 'snp_id', 'VARIANT_TYPE']
vars_bt = pbt.BedTool.from_dataframe(snv_indel[cols])
vars_bt = vars_bt.sort()
vars_bt = vars_bt.moveto(fn_vars_bt)

In [79]:
tx_info =  pd.read_table('/publicdata/gencode_v19_20151104/transcript_to_gene.tsv', names=['tx_id', 'gene_id']).pipe(prep_tx_info)
tx_gene_dict = tx_info.gene_id_mod.to_dict()

fn = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/enrichment_shuffle_svs/bed_files_features/genic_features_all.bed'
features_bt = pbt.BedTool(fn)

In [81]:
intersect_feat, info_all_feat = intersect_and_categorize_loc(vars_bt, features_bt, tx_gene_dict)
info_cat_per_gene_all =  post_process_intersect(info_all_feat, snv_indels_lead)

In [82]:
cat_sv_str_overall_sets =  genes_impacted_at_top_level(info_cat_per_gene_all)
cat_sv_str_overall_strings = genes_impacted_at_top_level(info_cat_per_gene_all, out_strings=True)

In [83]:
CM.save_dataframe('info_var_gene_intersects_snv_indel_lead', info_cat_per_gene_all, 
                  outdir, print_vars_recorded_loc=False)

info_var_gene_intersects_snv_indel_lead = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_intersects_snv_indel_lead.pkl')
info_var_gene_intersects_snv_indel_lead = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_intersects_snv_indel_lead.tsv', sep='\t')


In [84]:
CM.save_dataframe('info_var_gene_wide_string_snv_indel_lead', cat_sv_str_overall_strings, outdir,
                 print_vars_recorded_loc=False)

info_var_gene_wide_string_snv_indel_lead = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_string_snv_indel_lead.pkl')
info_var_gene_wide_string_snv_indel_lead = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_string_snv_indel_lead.tsv', sep='\t')


In [85]:
CM.save_dataframe('info_var_gene_wide_sets_snv_indel_lead', cat_sv_str_overall_sets, 
                  outdir, print_vars_recorded_loc=False)

info_var_gene_wide_sets_snv_indel_lead = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_sets_snv_indel_lead.pkl')
info_var_gene_wide_sets_snv_indel_lead = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_sets_snv_indel_lead.tsv', sep='\t')


In [90]:
info_var_gene_intersect_sv_str = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/info_var_gene_intersects_i2QTL_472_nnref1.pkl')


In [91]:
info_var_gene_wide_sv_str_string = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/consented_472/info_var_gene_wide_string_i2QTL_472_nnref1.pkl')

In [94]:
snv_lead_and_all_sv = pd.concat([info_var_gene_intersect_sv_str, info_cat_per_gene_all])

In [124]:
CM.save_dataframe('info_var_gene_intersects_sv_str_w_joint_leads' ,snv_lead_and_all_sv, outdir, print_vars_recorded_loc=False)

info_var_gene_intersects_sv_str_w_joint_leads = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_intersects_sv_str_w_joint_leads.pkl')
info_var_gene_intersects_sv_str_w_joint_leads = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_intersects_sv_str_w_joint_leads.tsv', sep='\t')


In [125]:
snv_lead_and_all_sv_gene_wide = pd.concat([info_var_gene_wide_sv_str_string, cat_sv_str_overall_strings])

In [126]:
CM.save_dataframe('info_var_gene_wide_sv_str_w_joint_leads' ,snv_lead_and_all_sv_gene_wide, outdir, print_vars_recorded_loc=False)

info_var_gene_wide_sv_str_w_joint_leads = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_sv_str_w_joint_leads.pkl')
info_var_gene_wide_sv_str_w_joint_leads = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_sv_str_w_joint_leads.tsv', sep='\t')


# All Significant SNV/INDEL

In [2]:
dy_name = 'qtl_results_01_17'
outdir = os.path.join(private_out, dy_name)
if not os.path.exists(outdir):
    DJ.makedir(outdir)

In [17]:
sig_joint_story5_annot = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/eqtl_processing_joint/qtl_results_01_19/sig_joint_story5_annot.pkl')

In [18]:
sig_joint_story5_annot['chrom'] = 'chr' + sig_joint_story5_annot.feature_chromosome.astype(str)

In [19]:
all_sig_vars = sig_joint_story5_annot.drop_duplicates(['snp_id'])

In [20]:
# all_sig_vars['start_variant_bed	end_variant'.split()].applymap(int)

In [21]:
fn_vars_bt = dy_bed_files + '/sig_snv_indel_leads_all.bed'
cols = ['chrom', 'start_variant_bed', 'end_variant', 'snp_id', 'VARIANT_TYPE']
vars_bt = pbt.BedTool.from_dataframe(all_sig_vars[cols])
vars_bt = vars_bt.sort()
vars_bt = vars_bt.moveto(fn_vars_bt)

In [22]:
tx_info =  pd.read_table('/publicdata/gencode_v19_20151104/transcript_to_gene.tsv', names=['tx_id', 'gene_id']).pipe(prep_tx_info)
tx_gene_dict = tx_info.gene_id_mod.to_dict()

fn = '/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/enrichment_shuffle_svs/bed_files_features/genic_features_all.bed'
features_bt = pbt.BedTool(fn)

In [27]:
all_sig_vars['ID'] = all_sig_vars.snp_id

In [28]:
intersect_feat, info_all_feat = intersect_and_categorize_loc(vars_bt, features_bt, tx_gene_dict)
info_cat_per_gene_all =  post_process_intersect(info_all_feat, all_sig_vars)

In [29]:
cat_sv_str_overall_sets =  genes_impacted_at_top_level(info_cat_per_gene_all)
cat_sv_str_overall_strings = genes_impacted_at_top_level(info_cat_per_gene_all, out_strings=True)

In [31]:
CM.save_dataframe('info_var_gene_intersects_joint_sig', info_cat_per_gene_all, 
                  outdir, print_vars_recorded_loc=False)

info_var_gene_intersects_joint_sig = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_intersects_joint_sig.pkl')
info_var_gene_intersects_joint_sig = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_intersects_joint_sig.tsv', sep='\t')


In [32]:
CM.save_dataframe('info_var_gene_wide_string_joint_sig', cat_sv_str_overall_strings, outdir,
                 print_vars_recorded_loc=False)

info_var_gene_wide_string_joint_sig = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_string_joint_sig.pkl')
info_var_gene_wide_string_joint_sig = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_string_joint_sig.tsv', sep='\t')


In [34]:
# accidentally oversaved the snv_indel_lead ones oops
CM.save_dataframe('info_var_gene_wide_sets_joint_sig', cat_sv_str_overall_sets, 
                  outdir, print_vars_recorded_loc=False)

info_var_gene_wide_sets_joint_sig = pd.read_pickle('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_sets_joint_sig.pkl')
info_var_gene_wide_sets_joint_sig = pd.read_csv('/frazer01/projects/hipsci/analysis/i2QTL-sv-analysis/private_output/categorize_sv_wrt_genes/qtl_results_01_17/info_var_gene_wide_sets_joint_sig.tsv', sep='\t')
